In [1]:
%load_ext autoreload
%autoreload 2

import bz2
import json

import pandas as pd

from helpers import *

In [4]:
# Read file in chunks
# with pd.read_json(DATA_FOLDER + 'quotes-2020-party_labeled.json', lines=True, chunksize=10000) as df_reader:
with pd.read_json(DATA_FOLDER + 'quotes-2020-party_labeled_small.json.bz2', lines=True, compression='bz2', chunksize=100000) as df_reader:
    for chunk in df_reader:
        break


df_quotes = chunk


In [5]:
df_quotes

,quoteID,quotation,speaker,date,numOccurrences,id,party_label
0,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,Sue Myrick,2020-01-16 12:00:13,1,Q367796,R
1,2020-01-26-000499,a few of the candidates who will do better in ...,Dave Loebsack,2020-01-26 13:21:36,11,Q771586,D
2,2020-01-26-040663,"The generational thing is important, quite hon...",Dave Loebsack,2020-01-26 13:21:36,11,Q771586,D
3,2020-01-20-000982,a host of other protections,Debbie Lesko,2020-01-20 15:32:48,1,Q16731415,R
4,2020-01-09-060095,"No, are you kidding? The way that this place l...",Debbie Lesko,2020-01-09 23:15:21,1,Q16731415,R
...,...,...,...,...,...,...,...
30523,2020-01-28-069262,She is ready to win and be an independent voic...,Cheri Bustos,2020-01-28 21:15:19,1,Q723148,D
30524,2020-04-15-075002,We're not used to a virus that we can't see.,Tom Corcoran,2020-04-15 00:00:00,1,Q7482898,R
30525,2020-02-12-019131,ensure my constituents have reliable access to...,Tammy Baldwin,2020-02-12 21:20:06,2,Q40628,D
30526,2020-01-11-021799,In terms of the actual investment and dollar s...,Scott Murphy,2020-01-11 08:00:00,1,Q290602,D
